In [2]:
# Mixed Model Evaluation
sigmaSqE=3;
sigmaSqG=1;
sigmaSqP=sigmaSqG+sigmaSqE;
heritability=sigmaSqG/sigmaSqP;
lambda=(1-heritability)/heritability;  # or lambda=sigmaSqE/sigmaSqG

3.0

In [3]:
# clone the Pedigree Module - only need to do this once
#Pkg.clone("https://github.com/reworkhow/PedModule.jl.git")
using PedModule
using Distributions

# Create your own pedigree here
pedigree = [1 0 0
            2 0 0
            3 0 0
            4 1 2 
            5 1 2 
            6 1 3]

writedlm("pedFile",pedigree)   # Save the pedigree file to disk



#Julia can execute unix commands!  Preface them with ;

In [4]:
;ls

1. Introduction.pdf
1. Introduction.pptx
2. Simple_Linear_Models.pptx
3. Pedigree BLUP.pptx
4. Fixed Effects Models.pptx
5. GBLUP and SIngleStep.pptx
7. Genomics_In_Beef_Industry Brief.pptx
7. Genomics_In_Beef_Industry.pptx
8a Haplotypes.pptx
8b Emre_Human_Data.pptx
8c GenomicPredictionUpperLimits.pptx
8d SingleStep.pptx
Agenda.docx
BLUPexample.jl
GBLUP_Answers
GBLUP_results
GWAS_Windows.pptx
Julia-cheatsheet.pdf
LS_MCMC_conceptual.txt
LinearRegressionMCMCwithUnknownResVar.jl
LinearRegressionMCMCwithknownResVar.jl
Math_Stat_foundations.pdf
MonteCarlo_BVN_integration_problem.pptx
PBLUP.ipynb
Problem 1.docx
Problem 1.pdf
Problem 2.docx
Problem 2.pdf
Project1_BLUPexample.jl
Project2_BLUPexample.jl
example.jl
interactive_MH_Gibbs.ipynb
pedFile
~$3. Pedigree BLUP.pptx


In [18]:
;cat pedFile

1	0	0
2	0	0
3	0	0
4	1	2
5	1	2
6	1	3


In [11]:
ped=PedModule.mkPed("pedFile")
ped.idMap
PedModule.getIDs(ped)         # Access the reordered IDs
Ainv=PedModule.AInverse(ped)  # Note this is a sparse matrix



# full() converts a sparse matrix to a dense matrix
# round() reduces the number of decimal places
A=round(inv(full(Ainv)),2)

numAnimals=size(Ainv,1)

#Create the incidence matrices
X=ones(numAnimals,1);
Z=eye(numAnimals);
numFixed=size(X,2);



In [6]:
srand(2)      # Set the integer seed for the random number generator
b=rand(Normal(0,sqrt(sigmaSqP)),numFixed);
d=MvNormal(zeros(numAnimals),A*sigmaSqG)
u=rand(d,1)
e=rand(Normal(0.0,sqrt(sigmaSqE)),numAnimals);

# Simulate data following a mixed model
y=X*b + Z*u + e

# You could make some observations randomly missing (delete rows of y, X, Z and e)



6x1 Array{Float64,2}:
  1.63208 
  3.03273 
  1.60903 
  0.16389 
 -0.20262 
  0.755514

In [12]:
# Setup the left- and right-hand side of Henderson's mixed model equations






mmeLhs=[X'X X'Z
        Z'X (Z'Z+Ainv*lambda)]
mmeRhs=[X'y
        Z'y]
 
nothing

In [8]:
# Now choose among various solvers
# Here we will use direct solution to get PEV
# Alternatives are PCG (iterative or indirect solution)
# MCMC (just like the code we already developed)        
                
mmeInv=inv(mmeLhs)
soln=mmeInv*mmeRhs
pev=(diag(mmeInv)*sigmaSqE)[(numFixed+1):end]
reliability=1-pev/sigmaSqG
accuracy=sqrt(reliability)


# Note this may be different to sqrt(heritability) due to 
# i) chance sampling that creates covariance between u and e
# ii) loss of information from the fixed effects 


6-element Array{Float64,1}:
 0.416827
 0.437806
 0.366332
 0.366332
 0.445287
 0.373439

In [13]:
# GLS solution

G=A*sigmaSqG
R=eye(numAnimals)*sigmaSqE
V=Z*G*Z' + R
Vinv=inv(V)
GLS_LHS=X'Vinv*X
GLS_RHS=X'Vinv*y
GLS_bhat=GLS_LHS\GLS_RHS

y_adj=y-X*GLS_bhat

# BLUP via GLS
GLS_uhat=G*Z'Vinv*(y_adj)
nothing

In [15]:
[soln[(numFixed+1):end] GLS_uhat]

6x2 Array{Float64,2}:
 -0.00448257  -0.00448257
  0.369457     0.369457  
  0.220255     0.220255  
  0.0138066    0.0138066 
 -0.364974    -0.364974  
 -0.216432    -0.216432  

In [16]:
# Linear functions (ie contrasts) of BLUP
K=[2 -1 -1 0 0 0    
   0  1 -1 0 0 0]  # Define the linear contrast
   
# BLUP of the linear contrast
BLUPlinfn=K*soln[numFixed+1:end]
   
# Variance of the linear constrasts
varKu=K*G*K'

# Variance of the BLUP of linear contrasts
varKuhat=K*(G-mmeInv[numFixed+1:end,numFixed+1:end])*K'

# Reliability of the linear contrasts
relKuhat=diag(varKuhat)./diag(varKu)
nothing

In [17]:
[BLUPlinfn varKu varKuhat relKuhat]

2x6 Array{Float64,2}:
 -0.598677   5.0  -1.0   3.75949   -0.777783  0.751899
  0.149202  -1.0   1.0  -0.777783   0.728351  0.728351